# 9.10. PROJECT. Определяем количество ботов на сайте
1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.
2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

In [1]:
import json 
import requests
import pandas as pd 
from pprint import pprint

Делаем запрос по API с помощью следующих параметров:

In [2]:
token='AQAAAAABeRTAAAdQDtX3WHMRtEaXg9r4JXfvEDQ'
headers = {'Authorization': 'OAuth ' + token}
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions': 'ym:s:lastsignTrafficSource,ym:s:hour',
          'date1': '2020-09-01',
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)

Получаем данные с помощью функции json(), примененной к запросу response. Создаем датафрейм metrika_df из данных статистики metrika_data['data']:

In [3]:
metrika_data = response.json()
metrika_data['data']
metrika_df = pd.DataFrame(metrika_data['data'])
display(metrika_df.head())

,dimensions,metrics
0,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[251.0, 24.70119522]"
1,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[240.0, 26.25]"
2,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[228.0, 25.87719298]"
3,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[227.0, 28.1938326]"
4,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[225.0, 24.88888889]"


Создаем  функцию getMetrikaDataInListOfDicts() для того, чтобы избавиться от вложенных словарей.

In [4]:
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

Преобразуем наши данные с помощью этой функции и запишем в переменную metrika_list_of_dicts:

In [5]:
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:3])

[{'ym:s:hour': '16:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 24.70119522,
  'ym:s:visits': 251.0},
 {'ym:s:hour': '15:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 26.25,
  'ym:s:visits': 240.0},
 {'ym:s:hour': '11:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 25.87719298,
  'ym:s:visits': 228.0}]


Создаем датафрейм metrika_df. Для удобства переименуем столбцы и сразу же добавим новый столбец с расчётом количества ботов:

In [6]:
metrika_df = pd.DataFrame(metrika_list_of_dicts)
metrika_df.columns=['TrafficSource','Hour','Visits','RobotPercent']
metrika_df['Robot'] =(metrika_df['Visits']*metrika_df['RobotPercent'])/100
display(metrika_df.head())

,TrafficSource,Hour,Visits,RobotPercent,Robot
0,Search engine traffic,16:00,251.0,24.701195,62.0
1,Search engine traffic,15:00,240.0,26.250000,63.0
2,Search engine traffic,11:00,228.0,25.877193,59.0
3,Search engine traffic,13:00,227.0,28.193833,64.0
4,Search engine traffic,12:00,225.0,24.888889,56.0


Для того чтобы подробнее рассмотреть получившийся датафрейм используем метод info:

In [7]:
display(metrika_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TrafficSource  107 non-null    object 
 1   Hour           107 non-null    object 
 2   Visits         107 non-null    float64
 3   RobotPercent   107 non-null    float64
 4   Robot          107 non-null    float64
dtypes: float64(3), object(2)
memory usage: 4.3+ KB


None

Как видим пропущенных значений нет ни в одном столбце (в каждом по 107 строк), а тип данных подходит для дальнейших расчетов, значит преоразования не требуются.

Для ответа на первый вопрос создаем сводную таблицу pivot_metrika:

In [8]:
pivot_metrika=metrika_df.pivot_table(values=['Visits','Robot'], index=['TrafficSource'], aggfunc='sum',margins=True)
pivot_metrika

,Robot,Visits
TrafficSource,,
Cached page traffic,1.0,6.0
Direct traffic,177.0,757.0
Internal traffic,3.0,10.0
Link traffic,31.0,126.0
Recommendation system traffic,0.0,2.0
Search engine traffic,739.0,3334.0
Social network traffic,11.0,80.0
All,962.0,4315.0


In [9]:
print ('Общее количество визитов: {}'.format((pivot_metrika.iloc[-1][1])))
print ('Общее количество роботов: {}'.format(round(pivot_metrika.iloc[-1][0])))
print ('Доля роботов в общем трафике: {:.2%}'.format(pivot_metrika.iloc[-1][0]/pivot_metrika.iloc[-1][1]))

Общее количество визитов: 4315.0
Общее количество роботов: 962
Доля роботов в общем трафике: 22.29%


# Выводы
В ходе расчетов было найдено общее количество визитов на сайт за исследуемый период - 4 315. Из них роботов - 962, что составляет 22% от общего количества. Таким образом, видим, что доля реальных пользователей гораздо больше ботов.

Для ответа на второй вопрос построим сводную таблицу с дополнительной разбивкой по времени, оставив только те строки, где визитов было больше 20, чтобы убрать непоказательные случаи, и отсортируем по убыванию доли роботов:

In [10]:
metrika_df_more_visits=metrika_df[metrika_df['Visits']>20]
pivot_metrika1=metrika_df_more_visits.pivot_table(values=['Visits','Robot','RobotPercent'], index=['TrafficSource','Hour'], aggfunc='sum').sort_values(by='RobotPercent',ascending=False)
pivot_metrika1.head(10)

Robot  RobotPercent  Visits
TrafficSource         Hour                              
Direct traffic        13:00   13.0     40.625000    32.0
                      09:00   12.0     32.432432    37.0
                      23:00    8.0     29.629630    27.0
Search engine traffic 13:00   64.0     28.193833   227.0
                      17:00   55.0     26.315789   209.0
                      15:00   63.0     26.250000   240.0
                      11:00   59.0     25.877193   228.0
                      14:00   58.0     25.777778   225.0
Direct traffic        19:00   10.0     25.000000    40.0
Search engine traffic 12:00   56.0     24.888889   225.0

# Выводы:
Было определено, что чаще всего в процентном соотношении роботы заходят через Direct traffic в 13:00 (40% роботов от общего числа визитов). А наибольшее количество роботов заходят через Search engine traffic в 13:00 и 15:00 (в количестве 64 и 63 соответственно).